In [1]:
import os
import sys
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
sys.path.append("../modules/bertwsi/")

from wsi.lm_bert import LMBert
from wsi.wsi import perform_wsi_on_ds_gen
from wsi.WSISettings import WSISettings

In [4]:
def prepare_gen_data(row):
    
    start_idx, end_idx = [int(idx) for idx in row["positions"].split('-')]
    
    pre = row["context"][:start_idx].rstrip()
    target = row["context"][start_idx : end_idx + 1]
    post = row["context"][end_idx + 1:].lstrip()
    inst_id = f"{row['word']}.n.{row['context_id']}"
    
    return pre, target, post, inst_id


def get_clusters(inst_id_to_sense):
    
    clusters = {
        sent_id: int(max(sent_senses, key=sent_senses.get).split('.')[-1]) + 1 for \
        sent_id, sent_senses in inst_id_to_sense.items()
    }
    
    return clusters

### init

In [5]:
settings = WSISettings(
    n_represents=15,
    n_samples_per_rep=20,
    cuda_device=-1,  # cpu
    debug_dir="debug",
    disable_tfidf=False,
    disable_lemmatization=False,
    run_name="active-rutenten",
    patterns=[
        ("{pre} {target} (or even {mask_predict}) {post}", 0.4),
        ("{pre} {target_predict} {post}", 0.4),
    ],
    min_sense_instances=2,
    bert_model="cointegrated/rubert-tiny",
    spacy_lang="../modules/spacy-ru/ru2",
    max_batch_size=10,
    prediction_cutoff=200,
    max_number_senses=7,
)

In [6]:
settings

WSISettings(n_represents=15, n_samples_per_rep=20, cuda_device=-1, debug_dir='debug', disable_tfidf=False, disable_lemmatization=False, run_name='active-rutenten', patterns=[('{pre} {target} (or even {mask_predict}) {post}', 0.4), ('{pre} {target_predict} {post}', 0.4)], min_sense_instances=2, bert_model='cointegrated/rubert-tiny', spacy_lang='../modules/spacy-ru/ru2', max_batch_size=10, prediction_cutoff=200, max_number_senses=7)

In [7]:
lm = LMBert(
    cuda_device=settings.cuda_device,
    bert_model=settings.bert_model,
    spacy_lang=settings.spacy_lang,
    max_batch_size=settings.max_batch_size,
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
lemmatizing vocab: 100%|██████████| 29564/29564 [00:12<00:00, 2339.55it/s]


In [8]:
if settings.debug_dir:
    if not os.path.exists(settings.debug_dir):
        os.makedirs(settings.debug_dir)

### prepare data

In [9]:
data = pd.read_csv(
    "../data/russe-wsi-kit/data/additional/active-rutenten/train.csv",
    sep="\t",
)

In [10]:
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,альбом,2,NaN,88-94,достаточно лишь колесиком мышки крутить вниз. ...
1,2,альбом,3,NaN,85-91,"выступал в составе команды с таким названием, ..."
2,3,альбом,2,NaN,81-87,". Работает так себе, поскольку функция заточен..."
3,4,альбом,3,NaN,84-89,одержала победу в двух из пяти номинаций: 'Луч...
4,5,альбом,3,NaN,83-88,встречи с Божественным. Вы испытаете ни с чем ...


In [11]:
gen = data.apply(prepare_gen_data, axis=1).to_list()

In [12]:
gen[:5]

[('достаточно лишь колесиком мышки крутить вниз. И если вы захотите увеличить фотографию в',
  'альбоме',
  ', то все следующие фотографии будут также отображаться в полноразмерном варианте',
  'альбом.n.1'),
 ('выступал в составе команды с таким названием, однако тогда бэнд не записал ни одного',
  'альбома',
  '. В прошлом году Плант снова собрал коллектив Band of Joy и записал с этими музыкантами',
  'альбом.n.2'),
 ('. Работает так себе, поскольку функция заточена под банальные фотки из семейного',
  'альбома',
  ', а-ля «я и Эйфелева башня», где люди стоят анфас в центре кадра и смотрят в камеру',
  'альбом.n.3'),
 ("одержала победу в двух из пяти номинаций: 'Лучший танцевальный хит' и “Танцевальный",
  'альбом',
  "года'. В марте 2010 года певица объявила, что начала работать над своим новым студийным",
  'альбом.n.4'),
 ('встречи с Божественным. Вы испытаете ни с чем не сравнимое блаженство, слушая этот',
  'альбом',
  'во время занятий йогой или принимая сеанс массажа. Эк...',


### word sense induction

In [13]:
inst_id_to_sense = perform_wsi_on_ds_gen(
    lm=lm,
    ds_name="active-rutenten",
    gen=gen,
    wsisettings=settings,
    print_progress=True,
)

predicting substitutes active-rutenten: 100%|██████████| 20/20 [02:53<00:00,  8.69s/it]

writing active-rutenten key file to debug/active-rutenten-active-rutenten.key


In [14]:
clusters = get_clusters(inst_id_to_sense)

In [15]:
data["predict_sense_id"] = data.apply(
    lambda row: clusters[f"{row['word']}.n.{row['context_id']}"],
    axis=1,
)

In [16]:
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,альбом,2,2,88-94,достаточно лишь колесиком мышки крутить вниз. ...
1,2,альбом,3,2,85-91,"выступал в составе команды с таким названием, ..."
2,3,альбом,2,2,81-87,". Работает так себе, поскольку функция заточен..."
3,4,альбом,3,2,84-89,одержала победу в двух из пяти номинаций: 'Луч...
4,5,альбом,3,2,83-88,встречи с Божественным. Вы испытаете ни с чем ...


In [17]:
data.to_csv(
    "predictions/bert_wsi_prediction.tsv",
    sep="\t",
    index=False,
)

### validate

In [18]:
!python3 ../data/russe-wsi-kit/evaluate.py predictions/bert_wsi_prediction.tsv

word	ari	count
альбом	-0.006816	450
анатомия	0.041973	95
базар	0.000000	90
балет	-0.046736	94
беда	0.000000	93
бездна	0.000000	87
билет	-0.031509	447
блок	-0.000509	206
блоха	0.205132	86
брак	0.036957	96
бритва	0.000000	85
будущее	0.018825	83
вешалка	-0.005139	390
вилка	0.085528	302
винт	-0.001541	358
галерея	-0.119827	24
горбуша	0.000000	93
горшок	0.001628	406
гроза	0.040910	95
группа	-0.002265	91
	0.008125	3671
